<a href="https://colab.research.google.com/github/jaredzrks0/DSAN5400_final_project/blob/main/DSAN_5400_Final_News_Recommender_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
# Install ColBERT and add to path

!git -C ColBERT/ pull || git clone https://github.com/stanford-futuredata/ColBERT.git > /dev/null 2>&1
import sys; sys.path.insert(0, 'ColBERT/')

fatal: cannot change to 'ColBERT/': No such file or directory


In [3]:
# Download the ColBERT Checkpoint
!wget "https://downloads.cs.stanford.edu/nlp/data/colbert/colbertv2/colbertv2.0.tar.gz"
!mkdir -p checkpoints  # Create the 'checkpoints' directory if it doesn't exist
!tar -xvzf colbertv2.0.tar.gz -C checkpoints

--2024-12-06 21:51:27--  https://downloads.cs.stanford.edu/nlp/data/colbert/colbertv2/colbertv2.0.tar.gz
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 405924985 (387M) [application/octet-stream]
Saving to: ‘colbertv2.0.tar.gz’

colbertv2.0.tar.gz  100%[===================>] 387.12M  5.02MB/s    in 72s     

2024-12-06 21:52:39 (5.41 MB/s) - ‘colbertv2.0.tar.gz’ saved [405924985/405924985]

colbertv2.0/
colbertv2.0/artifact.metadata
colbertv2.0/vocab.txt
colbertv2.0/tokenizer.json
colbertv2.0/special_tokens_map.json
colbertv2.0/tokenizer_config.json
colbertv2.0/config.json
colbertv2.0/pytorch_model.bin


In [4]:
try: # When on google Colab, let's install all dependencies with pip.
    import google.colab
    !pip install -U pip > /dev/null 2>&1
    !pip install fsspec==2024.9.0 > /dev/null 2>&1
    !pip install -e ColBERT/['faiss-gpu','torch'] > /dev/null 2>&1
    !pip install --upgrade torch torchvision torchaudio > /dev/null 2>&1
except Exception:
  import sys; sys.path.insert(0, 'ColBERT/')
  try:
    from colbert import Indexer, Searcher
  except Exception:
    print("If you're running outside Colab, please make sure you install ColBERT in conda following the instructions in our README. You can also install (as above) with pip but it may install slower or less stable faiss or torch dependencies. Conda is recommended.")
    assert False

In [5]:
# ColBERT Imports
import colbert
from colbert import Indexer, Searcher
from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert.data import Queries, Collection

In [8]:
import pandas as pd

df = pd.read_csv('collection.tsv', sep='\t', header=None)


In [21]:
# Clean the df to get just a list of strings
texts = df[0]
articles = [text.split('\t')[1] for text in texts]

In [22]:
nbits = 2   # encode each dimension with 2 bits
doc_maxlen = 300 # truncate passages at 300 tokens

index_name = f'dsan5400_project_nbits={nbits}'

In [27]:
checkpoint = 'colbert-ir/colbertv2.0'

with Run().context(RunConfig(nranks=1, experiment='DSAN5400')):  # nranks specifies the number of GPUs to use
    config = ColBERTConfig(doc_maxlen=doc_maxlen, nbits=nbits, kmeans_niters=4) # kmeans_niters specifies the number of iterations of k-means clustering; 4 is a good and fast default.
                                                                                # Consider larger numbers for small datasets.

    indexer = Indexer(checkpoint=checkpoint, config=config)
    indexer.index(name=index_name, collection=articles[:500], overwrite=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


artifact.metadata:   0%|          | 0.00/1.63k [00:00<?, ?B/s]



[Dec 06, 22:02:33] #> Creating directory /content/experiments/DSAN5400/indexes/dsan5400_project_nbits=2 


#> Starting...
#> Joined...


In [29]:
# To create the searcher using its relative name (i.e., not a full path), set
# experiment=value_used_for_indexing in the RunConfig.
with Run().context(RunConfig(experiment='DSAN5400')):
    searcher = Searcher(index=index_name, collection=articles[:500])


# If you want to customize the search latency--quality tradeoff, you can also supply a
# config=ColBERTConfig(ncells=.., centroid_score_threshold=.., ndocs=..) argument.
# The default settings with k <= 10 (1, 0.5, 256) gives the fastest search,
# but you can gain more extensive search by setting larger values of k or
# manually specifying more conservative ColBERTConfig settings (e.g. (4, 0.4, 4096)).

[Dec 06, 22:11:27] #> Loading codec...
[Dec 06, 22:11:27] Loading decompress_residuals_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


/content/ColBERT/colbert/utils/amp.py:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()
/content/ColBERT/colbert/indexing/codecs/residual.py:141: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use c

[Dec 06, 22:11:27] Loading packbits_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:1964: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


[Dec 06, 22:11:27] #> Loading IVF...
[Dec 06, 22:11:27] #> Loading doclens...


/content/ColBERT/colbert/search/index_loader.py:33: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ivf, ivf_lengths = torch.load(os.path.join(self.index_path, "ivf.pid.pt"), 

[Dec 06, 22:11:27] #> Loading codes and residuals...



  0%|          | 0/1 [00:00<?, ?it/s]/content/ColBERT/colbert/indexing/codecs/residual_embeddings.py:86: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(cod

In [45]:
# query = 'was the election rigged'
query = articles[15]
print(f"#> {query}")

# Find the top-3 passages for this query
results = searcher.search(query, k=3)

# Print out the top-k retrieved passages
for passage_id, passage_rank, passage_score in zip(*results):
  print(passage_id)
  print(f"\t [{passage_rank}] \t\t {passage_score:.1f} \t\t {searcher.collection[passage_id]}")

 Lee, who faces another sentencing hearing over perjury charges later this month, and his party have accused prosecutors of pursuing a politically motivated case against him.
15
 Lee, who faces another sentencing hearing over perjury charges later this month, and his party have accused prosecutors of pursuing a politically motivated case against him.
187
  
382
 US media reports have indicated that Biden, in his final days in office, plans to rush aid to Ukraine, in order to ensure its continued ability to defend itself from Russian invasion.


In [41]:
searcher.collection[passage_id]

'The African nation has been rocked by violence since its October 9 vote, won by the Frelimo party, which has been in power for almost 50 years.\r Police in Mozambique have fired tear gas in the capital Maputo to disperse people protesting against the government over alleged fraud in last month’s elections.\r The demonstration on Thursday was the biggest yet against the governing Frelimo party, which was declared the winner of the October 9 election, extending its 49-year rule.\r \r \r \r \r Large crowds of mostly young men barricaded streets with burning tyres and waved homemade signs in support of opposition leader Venancio Mondlane, who says the vote was rigged and had called for a week of protests culminating on Thursday.\r As the protesters marched towards the city centre, police tried to stop them by using tear gas.\r In an interview with AFP news agency, Mondlane, who claims to be the actual winner of the election, said it was a “crucial moment” for the country.\r “I feel that t

In [35]:
results

([414, 29, 60], [1, 2, 3], [23.359375, 22.140625, 21.0625])